In [3]:
from transformers import pipeline
import random

unmasker = pipeline('fill-mask', model='aubmindlab/bert-base-arabertv02')

input_text = "مش معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب"



Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Masked sentence-> مش معقول مفتي السعوديه اكبر دوله [MASK] سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب
Augmented text-> مش معقول مفتي السعوديه اكبر دوله بالعالم سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب


In [19]:
import pandas as pd
train = pd.read_csv("/kaggle/input/test-tweets/tweets_test.csv", encoding='utf-16')
train = train.iloc[:5]


In [23]:
train.head()


,id,tweet
0,958390144494694400,اخطر اعداء الامه منافقوها ينشغلون بتشويه مصلحي...
1,958390531867971584,حتي مارك اليهودي يحب مخرجات مءتمر الخوار الوطن...
2,958390679780016129,gcc تصوير جوي لحصن مرحب في خيبر الذي فتحه النب...
3,958390811514822656,خدمات الكنيسه ماشيين مع بعض بمبدا اليهود لا يع...
4,958390828682096640,ذكر الله المنافقين اكثر من اليهود لانهم يستعمل...


In [9]:
orig_text_list = input_text.split()
len_input = len(orig_text_list)
rand_idx = random.randint(1,len_input-2)
rand_idx2 = random.randint(1,len_input-2)

new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
new_mask_sent = ' '.join(new_text_list)
print("Masked sentence->",new_mask_sent)

augmented_text_list = unmasker(new_mask_sent)
augmented_text = augmented_text_list[0]['sequence']

augmented_text = augmented_text.split()

new_text_list = augmented_text[:rand_idx2] + ['[MASK]'] + augmented_text[rand_idx2:]
new_mask_sent = ' '.join(new_text_list)

augmented_text_list = unmasker(new_mask_sent)
augmented_text = augmented_text_list[0]['sequence']

print("Augmented text->",augmented_text)

Masked sentence-> مش [MASK] معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب
Augmented text-> مشي معقول مفتي السعوديه اكبر دوله سنيه بالعالم يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب


In [10]:
input_text = "مش معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب"

orig_text_list = input_text.split()
len_input = len(orig_text_list)
#Random index where we want to replace the word 
rand_idx = random.randint(1,len_input-1)
orig_word = orig_text_list[rand_idx]
new_text_list = orig_text_list.copy()
new_text_list[rand_idx] = '[MASK]'
new_mask_sent = ' '.join(new_text_list)
print("Masked sentence->",new_mask_sent)
#I went to [MASK] a movie in the theater

augmented_text_list = unmasker(new_mask_sent)
#To ensure new word and old word are not name
for res in augmented_text_list:
    if res['token_str'] != orig_word:
        augmented_text = res['sequence']
        break
print("Augmented text->",augmented_text)
#I went to watch a movie in the theater

Masked sentence-> مش معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه [MASK] لطفك بنا يارب
Augmented text-> مش معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه يارب لطفك بنا يارب


In [13]:
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')

input_text = "I went to see a movie in the theater"
input_length = len(input_text.split())
num_new_words = 10
output_length = input_length + num_new_words
gpt_output = generator(input_text, max_length=output_length, num_return_sequences=5)
augmented_text = gpt_output[0]['generated_text']
print("Augmented text->",augmented_text)
#I went to see a movie in the theater, and the director was

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Augmented text-> I went to see a movie in the theater. I knew him. I was afraid of this


In [14]:
input_text = "مش معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب"
input_length = len(input_text.split())
num_new_words = 10
output_length = input_length + num_new_words
gpt_output = generator(input_text, max_length=output_length, num_return_sequences=5)
augmented_text = gpt_output[0]['generated_text']
print("Augmented text->",augmented_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 103, but ``max_length`` is set to 27. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


Augmented text-> مش معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا ياربي


In [25]:
train = pd.read_csv('/kaggle/input/test-tweets-df/test_tweets.csv', encoding='utf-16')
train.head()

,id,tweet,hate
0,958390144494694400,اخطر اعداء الامه منافقوها ينشغلون بتشويه مصلحي...,1
1,958390531867971584,حتي مارك اليهودي يحب مخرجات مءتمر الخوار الوطن...,0
2,958390679780016129,gcc تصوير جوي لحصن مرحب في خيبر الذي فتحه النب...,0
3,958390811514822656,خدمات الكنيسه ماشيين مع بعض بمبدا اليهود لا يع...,0
4,958390828682096640,ذكر الله المنافقين اكثر من اليهود لانهم يستعمل...,0


In [26]:
import tensorflow as tf

df = pd.DataFrame()
# Iterate over the rows in the dataframe
for index, row in train.iterrows():
  #Encode the text input as a BERT input
    input_text = row['tweet']
    
    orig_text_list = input_text.split()
    len_input = len(orig_text_list)
    rand_idx = random.randint(1,len_input-2)
    rand_idx2 = random.randint(1,len_input-2)

    new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
    new_mask_sent = ' '.join(new_text_list)
    print("Masked sentence->",new_mask_sent)

    augmented_text_list = unmasker(new_mask_sent)
    augmented_text = augmented_text_list[0]['sequence']

    augmented_text = augmented_text.split()

    new_text_list = augmented_text[:rand_idx2] + ['[MASK]'] + augmented_text[rand_idx2:]
    new_mask_sent = ' '.join(new_text_list)

    augmented_text_list = unmasker(new_mask_sent)
    augmented_text = augmented_text_list[0]['sequence']
  # Add the generated text to the dataframe
    df = df.append({'tweet': augmented_text, 'hate': row['hate']}, ignore_index=True)

Masked sentence-> اخطر اعداء الامه منافقوها ينشغلون بتشويه مصلحيها ليضعفوهم ويسكتون عن مفسديها ليقووهم كانوا يسخرون من الصحابه ويسكتون عن يهود المدينه فلاجل ذلك حاربوا الصحوه لاجل هذا [MASK] حاربوا الصحوه
Masked sentence-> حتي مارك [MASK] اليهودي يحب مخرجات مءتمر الخوار الوطني قبل قليل ارسل لي رساله تم اعاده تغيير صورتك الشخصيه الي الصوره
Masked sentence-> gcc تصوير جوي لحصن مرحب في خيبر الذي فتحه النبي صلي الله عليه وسلم [MASK] في معركه خيبر وكان مقر اليهود بعد اجلاءهم من المدينه المنوره
Masked sentence-> خدمات [MASK] الكنيسه ماشيين مع بعض بمبدا اليهود لا يعاملون السامريين
Masked sentence-> ذكر الله المنافقين [MASK] اكثر من اليهود لانهم يستعملون وساءل شرعيه لهدم الاصول ويخفون علي العامه فاعين البسطاء علي الوساءل واعينهم علي الاهداف ف لاجل هذا حاربوا الصحوه
Masked sentence-> انتم المصافحون العملاء من تلطخت يده يوما بمصافحه اليهود لن يكون صهره انظف منه فالطيور علي اشكالها تقع [MASK] البلطجي جبران قزم العهد
Masked sentence-> استهداف السعوديه واضح ٠ المسلم يذهب سنويا للحج وشهريا للعمره وي

KeyboardInterrupt: 